In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

df = pd.read_csv('dados/final.csv',index_col=0)
df.head()

,qt_salas_utilizadas,qt_salas_utiliza_climatizadas,qt_equip_dvd,qt_equip_som,qt_equip_tv,qt_equip_lousa_digital,qt_equip_multimidia,qt_desktop_aluno,qt_comp_portatil_aluno,qt_tablet_aluno,...,in_prof_sim,in_prof_tec_sim,in_eja_sim,in_eja_fund_sim,in_eja_med_sim,in_esp_sim,in_esp_cc_sim,in_esp_ce_sim,tp_lingua_indigena,tp_lingua_portugues
11024968,15,15,1,1,3,3,10,12,0,0,...,0,0,0,0,0,0,0,0,0,1
11025638,44,31,1,2,21,0,15,22,15,98,...,0,0,0,0,0,1,1,0,0,1
11006773,25,0,0,1,11,0,6,25,0,0,...,0,0,1,0,1,1,1,0,0,1
11006889,17,17,4,4,4,0,5,12,0,0,...,0,0,0,0,0,1,1,0,0,1
11007168,18,18,0,0,3,2,10,4,7,4,...,0,0,0,0,0,1,1,0,0,1


In [3]:
import numpy as np
df = df.sample(frac=1)
X = df.drop(columns=['nota enem'])
y = df[['nota enem']]

print(X.shape, y.shape)

(19022, 218) (19022, 1)


In [4]:
from sklearn.preprocessing import RobustScaler

scaler_X, scaler_y = RobustScaler(), RobustScaler()
scaler_X.fit(X)
scaler_y.fit(y)

X_norm, y_norm = scaler_X.transform(X), scaler_y.transform(y)
X, y = pd.DataFrame(X_norm, columns=X.columns), pd.DataFrame(y_norm, columns=y.columns)

X, y = np.array(X), np.array(y)
print(X.shape, y.shape)

(19022, 218) (19022, 1)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.7, random_state=42)
print('Dados Treino', X_train.shape, y_train.shape)
print('Dados Teste', X_test.shape, y_test.shape)

Dados Treino (13315, 218) (13315, 1)
Dados Teste (5707, 218) (5707, 1)


XGBoost

In [6]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

In [7]:
%%time
model.fit(X_train, y_train)

CPU times: total: 3min 38s
Wall time: 21.4 s


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False, eta=0.1,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.100000001, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, ...)

In [8]:
y_pred = model.predict(X_test)
MAE = mean_absolute_error(y_pred=y_pred, y_true=y_test)
MSE = mean_squared_error(y_pred=y_pred, y_true=y_test)
R2 = r2_score(y_pred=y_pred, y_true=y_test)
print(f'MAE:{MAE}    MSE:{MSE}    R2:{R2}')

MAE:0.281036283722154    MSE:0.137836041456251    R2:0.8221685343549806


LGBM Light Gradient Boost Machine

In [9]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    'max_depth': 7,
    'num_leaves': 12,
    'num_iterations': 1000
}

In [10]:
%%time
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                callbacks=[lgb.early_stopping(stopping_rounds=5)])

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008663 seconds.
You can set `force_col_wise=true` to remove the overhead.
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[107]	valid_0's l2: 0.135402	valid_0's l1: 0.278058
CPU times: total: 2.72 s
Wall time: 315 ms


In [11]:
y_pred = gbm.predict(X_test)
MAE = mean_absolute_error(y_pred=y_pred, y_true=y_test)
MSE = mean_squared_error(y_pred=y_pred, y_true=y_test)
R2 = r2_score(y_pred=y_pred, y_true=y_test)
print(f'MAE:{MAE}    MSE:{MSE}    R2:{R2}')

MAE:0.2780584116288926    MSE:0.13540179413271902    R2:0.8253091191012681


Rede Neural

In [12]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model = MLPRegressor(hidden_layer_sizes=(20,7), activation='logistic',
    alpha=0.01, batch_size='auto',max_iter=5000,solver='adam',
    beta_1=0.9, beta_2=0.99, early_stopping=False)

In [13]:
%%time
model.fit(X_train, y_train)

CPU times: total: 51.3 s
Wall time: 8.73 s


MLPRegressor(activation='logistic', alpha=0.01, beta_2=0.99,
             hidden_layer_sizes=(20, 7), max_iter=5000)

In [14]:
y_pred = model.predict(X_test)
MAE = mean_absolute_error(y_pred=y_pred, y_true=y_test)
MSE = mean_squared_error(y_pred=y_pred, y_true=y_test)
R2 = r2_score(y_pred=y_pred, y_true=y_test)
print(f'MAE:{MAE}    MSE:{MSE}    R2:{R2}')

MAE:0.3000682326016962    MSE:0.16030709374272537    R2:0.7931771318127221
